In [ ]:
# Libraries required for Neural Network
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import time 
import copy
import torch
import torch.optim as optim
from torch.autograd import grad
from torch.autograd import Variable
import torch.nn as nn
dtype = torch.float

%matplotlib inline

In [ ]:
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

In [ ]:
device = 'cpu'
# For plots
plt.rcParams.update({'font.size':16})

### Define general functions

In [ ]:
# Define the sin() activation function
class mySin(torch.nn.Module):
    @staticmethod
    def forward(input):
        return torch.sin(input)

In [ ]:
# Calculate the derivative with auto-differention
def df_dx(x,f):
    pts = torch.ones(x.shape, dtype=dtype, device=torch.device(device))
    return grad([f],[x],grad_outputs=pts,create_graph=True)[0]

In [ ]:
# Stochastic perturbation of the evaluation points
# force t[0]=t0 & force points to be in the t-interval
def perturbation(grid,t0,tf,sig=0.5):
    delta_t = grid[1] - grid[0]
    noise = delta_t * torch.randn_like(grid) * sig
    t = grid + noise
    t.data[2] = torch.ones(1,1)*(-1)
    t.data[t<t0] = t0 - t.data[t<t0]
    t.data[t>tf] = 2*tf - t.data[t>tf]
    t.data[0] = torch.ones(1,1)*t0
    t.data[-1] = torch.ones(1,1)*tf
    t.requires_grad = False
    return t

In [ ]:
# Trial or parametric solution for boundary contitions 
def parametric_solution(t,nn,t0,x1,tf):
    N1,N2 = nn(t)
    # There are two parametric solutions, we take some that satisfy dirichlet conditions of radial part of hydrogen atom
    g= 1-torch.exp(-(t-tf))
    R_hat = g*N1
    return R_hat, N2

In [ ]:
# Radial differential equation or eigenvalue problem of hamiltonian operator
def hamiltonian_equation(r,R,E,V):
    dR_dr = df_dx(r,R)
    dR_drr = df_dx(r,dR_dr)
    f = dR_drr + dR_dr*2/r + (2*E+V)*R
    L = (f.pow(2)).mean()
    var_loss = (f.pow(2)).var()    # variance of f
    H_R = -1*dR_drr/2 + V*R
    return L, f, H_R, var_loss 

In [ ]:
# Gives the potential at each point r
def potential(r):
    l = 0
    rnp = r.data.numpy()     # points from tensor to numpy
    Vnp = 2/rnp - l*(l+1)/rnp**2     # potential over numpy arrays
    Vtorch = torch.from_numpy(Vnp) 
    return Vtorch

### Define Neural Network model

In [ ]:
class qNN(torch.nn.Module):
    def __init__(self, n_hl=10):      # neurons in hidden layer
        super(qNN,self).__init__()

        # Define the activation function
        self.activ_func = mySin()
        # Define layers
        self.E_in = torch.nn.Linear(1,1)
        self.Lin_1 = torch.nn.Linear(2,n_hl)
        self.Lin_2 = torch.nn.Linear(n_hl,n_hl)
        self.out = torch.nn.Linear(n_hl,1)

    def forward(self,t):
            
        In_1 = self.E_in(torch.ones_like(t))
        L1 = self.Lin_1(torch.cat((t,In_1),1))
        h1 = self.activ_func(L1)
        L2 = self.Lin_2(h1)
        h2 = self.activ_func(L2)
        out = self.out(h2)

        return out, -1*torch.abs(In_1)   # Negative energy because eigenvalues are negatives for hydrogen atom

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)

### Training of Neural Network model

In [ ]:
def training(t0, tf, x1, neurons, epochs, n_train, lr, minibatch_number=1, orthtime=2e4):
    '''        
        Inputs
        t0 -- left point on domain
        tf -- right point on domain
        x1 -- 
        neurons -- number of units at hidden layer
        epochs -- number of iterations of training
        n_train -- number of points between t0 & tf including them
        lr -- learning rate 
        minibatch_number -- size of minibatch for points on domain
        orthtime -- time for training one eigenstate

        Outputs 
        model_1 -- best model for schrodinger equation (lowest loss)
        loss_histories -- tuple of lists with histories of variables
        runTime -- time of training
        fc_ground -- model for ground state 
        fc_first_excited -- model for first excited state
        model_0 -- last trained model 
    '''
    
    par2 = 0
    model_0 = qNN(neurons)
    model_0 = model_0.to(device)
    model_1 = 0
    betas = [0.999,0.9999]
    optimizer = optim.Adam(model_0.parameters(), lr=lr, betas=betas)

    # List's to save variables
    Loss_history = []         ;         Llim = 1e+20    # Upper limit of loss
    nontriv_loss_history = []
    SE_loss_history = []
    En_history = []
    orth_losses = []
    fc_ground = 0
    fc_first_excited = 0

    di = (None, 1e+20)
    dic = {}
    for i in range(1000):
        dic[i] = di

    grid = torch.linspace(t0,tf,n_train).reshape(-1,1)

    # Training iterations
    TeP0 = time.time()
    walle = 0

    for tt in range(epochs):
        
        t = torch.abs(perturbation(grid,t0,tf,sig=0.03*tf)) + 1e-1

        # Batching
        batch_size = int(n_train/minibatch_number)
        batch_start, batch_end = 0, batch_size

        idx = np.random.permutation(n_train)
        t_b = t[idx]
        t_b.requires_grad = True
        t_f = t[-1]
        t_f = t_f.reshape(-1,1)
        t_f.requires_grad = True
        loss = 0
        
        # Reinitialize weights at orthtime
        if tt == orthtime:
            model_0.apply(weights_init)

        for nbatch in range(minibatch_number):
            # Batch time set
            t_mb = t_b[batch_start:batch_end].to(device)
            norm_loss_reg = 1.0   # regularization coefficient for normalization loss
            orth_loss_reg = 0.01    # regularization coefficient for orthogonality loss

            # Neural Network solutions
            nn, En = model_0(t_mb)
            En_history.append(En[0].data.tolist()[0])  # Append energy values from tensor to scalar (not list)

            psi = parametric_solution(t_mb, model_0, t0, x1, tf)[0].to(device)
            Pot = potential(t_mb.to(device))
            Ltot, f_red, H_psi, var_loss = hamiltonian_equation(t_mb, psi, En.to(device), Pot.to(device))
            Ltot *= 1
            SE_loss_history.append(Ltot)
            criteria_loss = Ltot

            # Normalization loss
            Ltot += norm_loss_reg*(n_train/(tf-t0)*1.0 - torch.sqrt(torch.dot(psi[:,0],psi[:,0]))).pow(2) 

            # Ortholoss after orthtime
            if tt > orthtime and tt < 2*orthtime:
                par2 = parametric_solution(t_mb, fc_ground, t0, x1, tf)[0]
                ortho_loss = orth_loss_reg*torch.sqrt(torch.dot(par2[:,0]*t_mb[:,0], psi[:,0]*t_mb[:,0]).pow(2))#/100
                orth_losses.append(ortho_loss)
                Ltot += ortho_loss
            elif tt >= 2*orthtime:
                par2 = parametric_solution[0](t_mb, fc_ground, t0, x1, tf)
                par3 = parametric_solution[0](t_mb, fc_first_excited, t0, x1, tf)
                ortho_loss = orth_loss_reg*torch.sqrt(torch.dot((par2[:,0] + par3[:,0])*t_mb[:,0], psi[:,0]*t_mb[:,0]).pow(2))#/10
                orth_losses.append(ortho_loss)
                Ltot += ortho_loss

            # Keep the log
            nontriv_loss_history.append(norm_loss_reg*(n_train/(tf-t0)*1.0 - torch.sqrt(torch.dot(psi[:,0],psi[:,0]))).pow(2))

            # Optimizer
            Ltot.backward(retain_graph=False)
            optimizer.step()
            loss += Ltot.to(device).data.numpy()
            optimizer.zero_grad()

            batch_start += batch_size
            batch_end += batch_size

        # Keep the loss function history
        Loss_history.append(loss)

        # Keep the best model (lowest loss) by using a deep copy
        if criteria_loss < Llim:
            model_1 = copy.deepcopy(model_0)
            Llim = criteria_loss
            if tt < orthtime:
                fc_ground = copy.deepcopy(model_0)
        
        E_bin = abs(En[0].data.tolist()[0]//0.01)
        if criteria_loss < dic[E_bin][1]:
            dic[E_bin] = (copy.deepcopy(model_0), criteria_loss, (t_mb, f_red, H_psi, psi))
        
        if tt > orthtime and tt < 2*orthtime:
            fc_first_excited = copy.deepcopy(model_0)

    TePf = time.time()
    runTime = TePf - TeP0
    loss_histories = (Loss_history, nontriv_loss_history, SE_loss_history, model_0, En_history, dic, orth_losses)
    return model_1, loss_histories, runTime, fc_ground, fc_first_excited, model_0

In [ ]:
# Training the model
t0 = 0.1
tf = 15
xBC1 = 0

n_train, neurons, epochs, lr, mb, orthtime = 300, 100, int(1e4), 8e-3, 1, 2e4
model1, loss_history1, runTime1, fc_ground, first_exc, last_exc  = training(t0,tf,xBC1,neurons,
                                                                                    epochs,n_train,lr,mb,orthtime)

In [ ]:
torch.save(fc_ground,'fc_ground')
torch.save(first_exc,'fc_first_exc')
torch.save(last_exc,'fc_last_exc')
torch.save(loss_history1,'loss_history1')
#np.savetxt('loss_histories',loss_history1,delimiter=',')

In [ ]:
print('Training time (minutes): ',runTime1/60)

# Loss funtion
plt.figure(figsize=(10,6))
plt.plot(loss_history1[0], '-b', alpha=0.8)
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Total loss')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Test the predicted solution
nTest = n_train
tTest = torch.linspace(t0-0.1,tf+0.1,nTest).reshape(-1,1)
tTest.requires_grad = True
t_net = tTest.detach().numpy()
psi = parametric_solution(tTest, fc_ground.to(device), t0, xBC1, tf)[0]
psi_net = psi.data.numpy()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(t_net, psi_net, '-b', linewidth=1, alpha=0.8, label='ANN')
plt.xlabel('x')
plt.ylabel('$\psi(x)$')
plt.grid()
plt.tight_layout()
plt.show()